In [8]:
from pyspark.sql import SparkSession

In [9]:
spark = SparkSession.builder.appName('vizualizações3').getOrCreate()

In [10]:
df_parquet = spark.read.parquet('/home/rodrigo/Documentos/projetos/open_weather_api_apache/analise_dados_youtube/data/projetos_youtube_v2/ouro/assunto_cities_skylines_2/total_visualizacoes_por_semana/total_visualizacoes_por_semana.parquet')
df_parquet.createOrReplaceTempView('TOTAL_VISUALIZACOES')

In [14]:
spark.sql('SELECT * FROM TOTAL_VISUALIZACOES').show()

+---------------+-----------+--------------------+---------------------+--------------------+----------+-------------------+-----------------+-----------+-----------------------+---------------------+---------------+-------------+--------------------+
|       NM_CANAL|   ID_VIDEO|        TITULO_VIDEO|TOTAL_CARACTERE_VIDEO|                TAGS|TOTAL_TAGS|TOTAL_VISUALIZACOES|TOTAL_COMENTARIOS|TOTAL_LIKES|TOTAL_VISUALIZACOES_DIA|TOTAL_COMENTARIOS_DIA|TOTAL_LIKES_DIA|data_extracao|            ID_CANAL|
+---------------+-----------+--------------------+---------------------+--------------------+----------+-------------------+-----------------+-----------+-----------------------+---------------------+---------------+-------------+--------------------+
|ChratosGameplay|-QaSHAsHQSs|CITIES SKYLINES 2...|                   81|                null|         0|                  0|                0|         50|                      0|                    0|             50|   2023-10-27|UCrOH1V-FyMunB

In [15]:
spark.sql('SELECT * FROM TOTAL_VISUALIZACOES where data_extracao = "2023-10-25"').show()

+---------------+-----------+--------------------+---------------------+--------------------+----------+-------------------+-----------------+-----------+-----------------------+---------------------+---------------+-------------+--------------------+
|       NM_CANAL|   ID_VIDEO|        TITULO_VIDEO|TOTAL_CARACTERE_VIDEO|                TAGS|TOTAL_TAGS|TOTAL_VISUALIZACOES|TOTAL_COMENTARIOS|TOTAL_LIKES|TOTAL_VISUALIZACOES_DIA|TOTAL_COMENTARIOS_DIA|TOTAL_LIKES_DIA|data_extracao|            ID_CANAL|
+---------------+-----------+--------------------+---------------------+--------------------+----------+-------------------+-----------------+-----------+-----------------------+---------------------+---------------+-------------+--------------------+
|ChratosGameplay|-QaSHAsHQSs|CITIES SKYLINES 2...|                   81|                null|         0|                  0|                0|         50|                      0|                    0|             50|   2023-10-25|UCrOH1V-FyMunB

# Relação Comentários X Likes (proporção)

# Relação Visualizações X Likes (proporção)

# Dia em que os vídeos são Públicados com alta frequência

# Total de Tags X Vísualizações 

# Tamanho dos títulos do Vídeos X Vísualizações

# Total Víews por Vídeo

# A duração do vídeo é importante para visualizações e interação (curtidas/comentários)?